## This code identifies binary traits

In [1]:
!pip install openpyxl --no-index

Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2023/x86-64-v3, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2023/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/openpyxl-3.1.2+computecanada-py2.py3-none-any.whl
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/et_xmlfile-1.1.0+computecanada-py3-none-any.whl (from openpyxl)


In [1]:
import pandas as pd
import numpy as np
import csv
import openpyxl

In [2]:
df = pd.read_csv("/lustre06/project/6060121/CLSA_PheWeb_shared/Original/23ME002_UdeM_SGTaliun_Baseline/23ME002_UdeM_SGTaliun_Baseline_CoPv7_Qx_PA_BS.csv", low_memory = False)

In [4]:
phenotype_class = pd.read_excel("./2109005_McGill_DTaliun_Baseline_CoPv7-dictionary.xlsx")

In [7]:
# Find columns that only contain 0 and 1 values and possibly missing values
binary_columns = [col for col in df.columns if all((df[col].isin([0, 1])) | (df[col].isnull()))]
len(binary_columns)

1026

In [3]:
# binary_columns = [
#     col for col in df.columns 
#     if all((df[col].isin([0, 1])) | (df[col].isnull())) or all((df[col].isin([1, 2])) | (df[col].isnull()))
# ]
# len(binary_columns)

1094

In [8]:
# Load the second dataset
#phenotype_classes_df = phenotype_class
phenotype_class = phenotype_class[['name','class']]

phenotype_class['class'] = phenotype_class['class'].str.lower()

# Check the class(es) for each binary phenotype
binary_phenotype_classes = phenotype_class[phenotype_class['name'].isin(binary_columns)][['name','class']]
binary_phenotype_classes.to_excel("binary_phenotype_classes.xlsx", index=False)

### Generating Binary_Pheno.csv which contains the binary phenotypes and cases,controls,male,female

In [9]:
# Create a dictionary to store the results
results = {'Column': [], 'Cases': [], 'Controls': [], 'Total_case_control': [], 'Missing_values': [],
          'Male_Controls': [], 'Female_Controls': [], 'Male_Cases': [], 'Female_Cases': [],
           'Total_female': [], 'Total_male': [], 'Label': []
           }

for column in binary_columns:
    cases = (df[column] == 1).sum()
    controls = (df[column] == 0).sum()
    male_cases = ((df[column] == 1) & (df['SEX_ASK_COM'] == 'M')).sum()
    female_cases = ((df[column] == 1) & (df['SEX_ASK_COM'] == 'F')).sum()
    male_controls = ((df[column] == 0) & (df['SEX_ASK_COM'] == 'M')).sum()
    female_controls = ((df[column] == 0) & (df['SEX_ASK_COM'] == 'F')).sum()
    male_total = male_cases + male_controls
    female_total = female_cases + female_controls
    
    #finding the assigned label for each phenotype from the phenotype_class file
    index = phenotype_class[phenotype_class['name'] == column].index[0]
    label = phenotype_class['class'].iloc[index]
    

    results['Column'].append(column)
    results['Cases'].append(cases)
    results['Controls'].append(controls)
    results['Total_case_control'].append(controls+cases)
    results['Missing_values'].append(len(df) - (cases+controls))
    results['Male_Controls'].append(male_controls)
    results['Female_Controls'].append(female_controls)
    results['Male_Cases'].append(male_cases)
    results['Female_Cases'].append(female_cases)
    results['Total_female'].append(female_total)
    results['Total_male'].append(male_total)
    results['Label'].append(label)
    
     
    
    if male_total == 0 and female_total == 0:
        continue

    
    
# Convert the results to a Pandas dataframe
results_df = pd.DataFrame(results)

# Keep the ones with more than 1000 sample size
# results_df = results_df[results_df["Total_case_control"] >= 1000]

# Write the results to a CSV file
results_df.to_csv('Binary_Pheno.csv', index=False)

## testing different thresholds for cases

In [ ]:
# import pandas as pd

# # Load the CSV file
# df = pd.read_csv('Binary_Pheno.csv')

# # Filter rows where 'Cases' is less than or equal to 1000
# filtered_df = df[df['Cases'] <= 1000]

# # Print the number of rows with 'Cases' <= 1000
# print(f"Number of rows with 'Cases' <= 1000: {len(filtered_df)}")

# # Get the distribution of the 'Label' column in the filtered DataFrame
# label_distribution = filtered_df['Label'].value_counts()

# # Print the distribution of 'Label'
# print("Distribution of 'Label' for rows where 'Cases' <= 1000:")
# print(label_distribution)
# # we remove these

Number of rows with 'Cases' <= 1000: 744
Distribution of 'Label' for rows where 'Cases' <= 1000:
Label
behaviour         315
health            246
identity           69
socio-economic     43
measurements       42
medication         17
medications        12
Name: count, dtype: int64


In [18]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('Binary_Pheno.csv')

# List of cutoff values
cutoffs = range(1000, 0, -100)

# Prepare a list to store the summary data
summary_data = []

# Loop through each cutoff value
for cutoff in cutoffs:
    # Filter rows where 'Cases' is less than or equal to the current cutoff
    filtered_df = df[df['Cases'] <= cutoff]

    # Get the distribution of the 'Label' column in the filtered DataFrame
    label_distribution = filtered_df['Label'].value_counts().to_dict()

    # Store results in the summary list as a dictionary
    summary_data.append({
        'Cutoff': cutoff,
        'Num variables to remove': len(filtered_df),
        **label_distribution  # Expand the label distribution into the row dictionary
    })

# Convert summary data into a DataFrame for better visualization
summary_df = pd.DataFrame(summary_data)

# Display the DataFrame in the notebook
summary_df


,Cutoff,Num variables to remove,behaviour,health,identity,socio-economic,measurements,medication,medications
0,1000,744,315,246,69,43,42,17,12
1,900,734,312,243,67,41,42,17,12
2,800,724,310,239,66,38,42,17,12
3,700,712,307,233,65,36,42,17,12
4,600,698,301,229,64,33,42,17,12
5,500,685,299,222,63,30,42,17,12
6,400,652,289,213,60,22,40,17,11
7,300,626,281,202,55,21,39,17,11
8,200,579,266,188,48,13,36,17,11
9,100,490,234,151,34,9,34,17,11
